In [37]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD
from keras.constraints import maxnorm

In [73]:
#ModelCreate#Applies to GS1, GS2, GS4, GS5
def create_model(input_dim=13, optimizer='adam', init_mode='uniform', activation='relu', dropout_rate=-1, weight_constraint=-1, neurons=16):
	# create model
	model = Sequential()
	model.add(Dense(neurons, input_dim=input_dim, kernel_initializer=init_mode, activation=activation))        
	model.add(Dense(1, kernel_initializer=init_mode, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
	return model

In [49]:
##GS3
#def create_model3(input_dim=13, learn_rate=0.01, momentum=0):
#	# create model
#	model = Sequential()
#	model.add(Dense(16, input_dim=input_dim, activation='relu'))
#	model.add(Dense(1, activation='sigmoid'))
#	# Compile model
#	optimizer = SGD(lr=learn_rate, momentum=momentum)
#	model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
#	return model

In [62]:
def gs1Spec(input_dim=13):
    model = KerasClassifier(build_fn=create_model, input_dim=input_dim, verbose=0)
    # define the grid search parameters
    batch_size = [10, 20, 40, 60, 80, 100]
    epochs = [10, 50, 100]
    param_grid = dict(batch_size=batch_size, epochs=epochs)
    return model, batch_size, epochs, param_grid

In [63]:
def gs2Spec(input_dim=13, batch_size=10, epochs=100):
    #####GS2
    # create model
    model = KerasClassifier(build_fn=create_model, input_dim=input_dim, verbose=0, batch_size=batch_size, epochs=epochs)

    # define the grid search parameters
    optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
    param_grid = dict(optimizer=optimizer)
    
    return model, optimizer, param_grid

In [64]:
#GS3 is solely for testing SGD, and affects no other tests.
#def gs3Spec(input_dim=13, batch_size=10, epochs=100):
#    # create model
#    model = KerasClassifier(build_fn=create_model3, input_dim=input_dim, verbose=0, batch_size=batch_size, epochs=epochs)

    # define the grid search parameters
#    learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
#    momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
#    param_grid = dict(learn_rate=learn_rate, momentum=momentum)
#    return model, learn_rate, momentum, param_grid

In [65]:
def gs4Spec(input_dim=13, batch_size=10, epochs=100, optimizer='adam'):
    # create model
    model = KerasClassifier(build_fn=create_model, input_dim=input_dim, optimizer=optimizer, verbose=0, batch_size=batch_size, epochs=epochs)
    init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
    param_grid = dict(init_mode=init_mode)
    return model, init_mode, param_grid

In [66]:
def gs5Spec(input_dim=13, batch_size=10, epochs=100, optimizer='adam', init_mode='uniform'):
    # create model
    model = KerasClassifier(build_fn=create_model, input_dim=input_dim, optimizer=optimizer, init_mode=init_mode, verbose=0, batch_size=batch_size, epochs=epochs)
    activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
    param_grid = dict(activation=activation)
    
    return model, activation, param_grid

In [67]:
def gs6Spec(input_dim=13, batch_size=10, epochs=100, optimizer='adam', init_mode='uniform', activation='relu'):
    # create model
    model = KerasClassifier(build_fn=create_model, input_dim=input_dim, optimizer=optimizer, init_mode=init_mode, activation=activation, dropout_rate=0.0, weight_constraint=0, verbose=0, batch_size=batch_size, epochs=epochs)
    weight_constraint = [1, 2, 3, 4, 5]
    dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    param_grid = dict(dropout_rate=dropout_rate, weight_constraint=weight_constraint)
    
    return model, weight_constraint, dropout_rate, param_grid

In [68]:
def gs7Spec(input_dim=13, batch_size=10, epochs=100, optimizer='adam', init_mode='uniform', activation='relu', dropout_rate=0.0, weight_constraint=0):
    # create model
    model = KerasClassifier(build_fn=create_model, input_dim=input_dim, optimizer=optimizer, init_mode=init_mode, activation=activation, dropout_rate=dropout_rate, weight_constraint=weight_constraint, verbose=0, batch_size=batch_size, epochs=epochs)
    neurons = [1, 5, 10, 15, 20, 25, 30]
    param_grid = dict(neurons=neurons)
    return model, neurons, param_grid

In [69]:
def studentSetup():
    ####Education Data Setup
    # load dataset
    dataset = pd.read_csv("slim-xAPI-Edu-Data.csv")
    #students failed if they are in class L.
    dataset['Failed'] = np.where(dataset['Class'] == 'L', True, False)
    dataset['gender'] = np.where(dataset['gender']=='M',1,0)
    dataset['Relation'] = np.where(dataset['Relation']=='Father',1,0)
    dataset['ParentAnsweringSurvey'] = np.where(dataset['ParentAnsweringSurvey'] == 'Yes', 1, 0)
    dataset['ParentschoolSatisfaction'] = np.where(dataset['ParentschoolSatisfaction'] == 'Yes', 1, 0)
    dataset['AbsentMoreThanWeek'] = np.where(dataset['StudentAbsenceDays'] == 'Above-7', 1, 0)
    dataset['Semester'] = np.where(dataset['Semester'] == 'F', 1, 0)

    X = dataset[['raisedhands', 'VisITedResources', 'SectionID', 'Topic', 'StageID', 'AnnouncementsView', 'Semester', 'Discussion', 'gender', 'Relation', 'ParentAnsweringSurvey', 'ParentschoolSatisfaction', 'AbsentMoreThanWeek']]
    Y = dataset[['Failed']]
    
    return X, Y

In [70]:
def colourSetup():
    dataset = pd.read_csv("colour-data.csv")
    X = dataset[['R', 'G', 'B']]
    Y = dataset[['Label']]
    return X, Y

In [86]:
def weatherSetup():
    dataset = pd.read_csv("modified-monthly-data-labelled.csv")
    X = dataset.iloc[:,0:61]
    Y = dataset.iloc[:,61]
    return X, Y

In [71]:
def printResult(grid_result):
        #Printing Results / Best Parameters.
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))

In [11]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [12]:
X, Y = studentSetup()

In [74]:
model, batch_size, epochs, param_grid = gs1Spec(input_dim=X.shape[1])

In [46]:
#####Processing Search 1 - for Batch Size and Epochs.
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(np.array(X), np.array(Y))

In [47]:
bestBatchSize = grid_result.best_params_.get('batch_size')
bestEpochs = grid_result.best_params_.get('epochs')
printResult(grid_result)

Best: 0.910417 using {'batch_size': 20, 'epochs': 100}
0.841667 (0.025685) with: {'batch_size': 10, 'epochs': 10}
0.900000 (0.000000) with: {'batch_size': 10, 'epochs': 50}
0.908333 (0.005893) with: {'batch_size': 10, 'epochs': 100}
0.845833 (0.017922) with: {'batch_size': 20, 'epochs': 10}
0.887500 (0.018399) with: {'batch_size': 20, 'epochs': 50}
0.910417 (0.010623) with: {'batch_size': 20, 'epochs': 100}
0.802083 (0.055979) with: {'batch_size': 40, 'epochs': 10}
0.881250 (0.026516) with: {'batch_size': 40, 'epochs': 50}
0.902083 (0.010623) with: {'batch_size': 40, 'epochs': 100}
0.789583 (0.069472) with: {'batch_size': 60, 'epochs': 10}
0.864583 (0.029463) with: {'batch_size': 60, 'epochs': 50}
0.889583 (0.026187) with: {'batch_size': 60, 'epochs': 100}
0.754167 (0.056211) with: {'batch_size': 80, 'epochs': 10}
0.847917 (0.020624) with: {'batch_size': 80, 'epochs': 50}
0.868750 (0.020412) with: {'batch_size': 80, 'epochs': 100}
0.743750 (0.051791) with: {'batch_size': 100, 'epochs':

In [19]:
#####Processing Search 2 - for best Optimizer.
model, optimizer, param_grid = gs2Spec(input_dim=X.shape[1], batch_size=bestBatchSize, epochs=bestEpochs)

In [20]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(np.array(X), np.array(Y))

In [21]:
#From GS2, we get the best optimizer.  Save this for future reference.  Pray it is not SGD.
bestOptimizer = grid_result.best_params_.get('optimizer')
printResult(grid_result)

Best: 0.916667 using {'optimizer': 'Adam'}
0.777083 (0.074361) with: {'optimizer': 'SGD'}
0.908333 (0.010623) with: {'optimizer': 'RMSprop'}
0.897917 (0.015590) with: {'optimizer': 'Adagrad'}
0.904167 (0.015590) with: {'optimizer': 'Adadelta'}
0.916667 (0.014731) with: {'optimizer': 'Adam'}
0.910417 (0.005893) with: {'optimizer': 'Adamax'}
0.893750 (0.020412) with: {'optimizer': 'Nadam'}


In [22]:
#model, learn_rate, momentum, param_grid = gs3Spec(input_dim=X.shape[1], batch_size=bestBatchSize, epochs=bestEpochs)

In [23]:
#grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
#grid_result = grid.fit(np.array(X), np.array(Y))

In [24]:
##we get no useful info from gs3 except what would be best to use for sgd
#bestSGDLearn = grid_result.best_params_.get('learn_rate')
#bestSGDMomentum = grid_result.best_params_.get('momentum')

#printResult(grid_result)

Best: 0.868750 using {'learn_rate': 0.001, 'momentum': 0.8}
0.858333 (0.029463) with: {'learn_rate': 0.001, 'momentum': 0.0}
0.858333 (0.043401) with: {'learn_rate': 0.001, 'momentum': 0.2}
0.850000 (0.018399) with: {'learn_rate': 0.001, 'momentum': 0.4}
0.835417 (0.026187) with: {'learn_rate': 0.001, 'momentum': 0.6}
0.868750 (0.038528) with: {'learn_rate': 0.001, 'momentum': 0.8}
0.862500 (0.005103) with: {'learn_rate': 0.001, 'momentum': 0.9}
0.756250 (0.058852) with: {'learn_rate': 0.01, 'momentum': 0.0}
0.789583 (0.057358) with: {'learn_rate': 0.01, 'momentum': 0.2}
0.847917 (0.028106) with: {'learn_rate': 0.01, 'momentum': 0.4}
0.704167 (0.093030) with: {'learn_rate': 0.01, 'momentum': 0.6}
0.808333 (0.095516) with: {'learn_rate': 0.01, 'momentum': 0.8}
0.775000 (0.058852) with: {'learn_rate': 0.01, 'momentum': 0.9}
0.735417 (0.044585) with: {'learn_rate': 0.1, 'momentum': 0.0}
0.735417 (0.044585) with: {'learn_rate': 0.1, 'momentum': 0.2}
0.618750 (0.208104) with: {'learn_rate':

In [53]:
model, init_mode, param_grid = gs4Spec(input_dim=X.shape[1], batch_size=bestBatchSize, epochs=bestEpochs, optimizer=bestOptimizer)

In [54]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(np.array(X), np.array(Y))

In [69]:
#from gs4, we get the best init mode.
bestInitMode = grid_result.best_params_.get('init_mode')
printResult(grid_result)

Best: 0.908333 using {'init_mode': 'glorot_uniform'}
0.906250 (0.010206) with: {'init_mode': 'uniform'}
0.904167 (0.014731) with: {'init_mode': 'lecun_uniform'}
0.889583 (0.007795) with: {'init_mode': 'normal'}
0.735417 (0.044585) with: {'init_mode': 'zero'}
0.897917 (0.002946) with: {'init_mode': 'glorot_normal'}
0.908333 (0.011785) with: {'init_mode': 'glorot_uniform'}
0.893750 (0.023385) with: {'init_mode': 'he_normal'}
0.877083 (0.029017) with: {'init_mode': 'he_uniform'}


In [74]:
model, activation, param_grid = gs5Spec(input_dim=X.shape[1], batch_size=bestBatchSize, epochs=bestEpochs, optimizer=bestOptimizer, init_mode=bestInitMode)

In [75]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(np.array(X), np.array(Y))

In [76]:
bestActivation = grid_result.best_params_.get('activation')
printResult(grid_result)

Best: 0.904167 using {'activation': 'linear'}
0.885417 (0.033978) with: {'activation': 'softmax'}
0.887500 (0.031041) with: {'activation': 'softplus'}
0.877083 (0.012843) with: {'activation': 'softsign'}
0.893750 (0.008839) with: {'activation': 'relu'}
0.870833 (0.005893) with: {'activation': 'tanh'}
0.883333 (0.010623) with: {'activation': 'sigmoid'}
0.872917 (0.026187) with: {'activation': 'hard_sigmoid'}
0.904167 (0.010623) with: {'activation': 'linear'}


In [15]:


#model, weight_constraint, dropout_rate, param_grid = gs6Spec(input_dim=X.shape[1], batch_size=bestBatchSize, epochs=bestEpochs, optimizer=bestOptimizer, init_mode=bestInitMode, activation=bestActivation)

In [16]:
#grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
#grid_result = grid.fit(np.array(X), np.array(Y))

In [27]:
#Pick these up for later print-off.
#bestDropoutRate = grid_result.best_params_.get('dropout_rate')
#bestWeightConstraint = grid_result.best_params_.get('weight_constraint')
#printResult(grid_result)

#bestBatchSize = 20
#bestEpochs = 100
#bestOptimizer = 'Adam'
#bestInitMode = 'glorot_uniform'
#bestActivation = 'linear'
#bestDropoutRate = 0
#bestWeightConstraint = 0
#bestNeurons = 30

In [28]:
model, neurons, param_grid = gs7Spec(input_dim=X.shape[1], batch_size=bestBatchSize, epochs=bestEpochs, optimizer=bestOptimizer, init_mode=bestInitMode, dropout_rate = 0, weight_constraint=0)

In [29]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(np.array(X), np.array(Y))

In [30]:
bestNeurons = grid_result.best_params_.get('neurons')
printResult(grid_result)

Best: 0.904167 using {'neurons': 30}
0.789583 (0.036204) with: {'neurons': 1}
0.862500 (0.015309) with: {'neurons': 5}
0.897917 (0.002946) with: {'neurons': 10}
0.897917 (0.016404) with: {'neurons': 15}
0.893750 (0.027003) with: {'neurons': 20}
0.902083 (0.015590) with: {'neurons': 25}
0.904167 (0.020624) with: {'neurons': 30}


In [75]:
X, Y = colourSetup()

In [77]:
model, batch_size, epochs, param_grid = gs1Spec(input_dim=X.shape[1])

In [78]:
#####Processing Search 1 - for Batch Size and Epochs.
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(np.array(X), np.array(Y))

In [79]:
bestBatchSize = grid_result.best_params_.get('batch_size')
bestEpochs = grid_result.best_params_.get('epochs')
printResult(grid_result)

Best: 0.149841 using {'batch_size': 10, 'epochs': 100}
0.141892 (0.019090) with: {'batch_size': 10, 'epochs': 10}
0.145866 (0.015040) with: {'batch_size': 10, 'epochs': 50}
0.149841 (0.012299) with: {'batch_size': 10, 'epochs': 100}
0.141892 (0.019090) with: {'batch_size': 20, 'epochs': 10}
0.143879 (0.016953) with: {'batch_size': 20, 'epochs': 50}
0.147854 (0.013447) with: {'batch_size': 20, 'epochs': 100}
0.141892 (0.019090) with: {'batch_size': 40, 'epochs': 10}
0.142687 (0.018213) with: {'batch_size': 40, 'epochs': 50}
0.144674 (0.016157) with: {'batch_size': 40, 'epochs': 100}
0.141892 (0.019090) with: {'batch_size': 60, 'epochs': 10}
0.142687 (0.018213) with: {'batch_size': 60, 'epochs': 50}
0.142687 (0.018213) with: {'batch_size': 60, 'epochs': 100}
0.141892 (0.019090) with: {'batch_size': 80, 'epochs': 10}
0.141892 (0.019090) with: {'batch_size': 80, 'epochs': 50}
0.143084 (0.017786) with: {'batch_size': 80, 'epochs': 100}
0.141892 (0.019090) with: {'batch_size': 100, 'epochs':

In [80]:
#####Processing Search 2 - for best Optimizer.
model, optimizer, param_grid = gs2Spec(input_dim=X.shape[1], batch_size=bestBatchSize, epochs=bestEpochs)

In [81]:
#####Processing Search 1 - for Batch Size and Epochs.
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(np.array(X), np.array(Y))

In [82]:
#From GS2, we get the best optimizer.  Save this for future reference.  Pray it is not SGD.
bestOptimizer = grid_result.best_params_.get('optimizer')
printResult(grid_result)

Best: 0.155803 using {'optimizer': 'Nadam'}
0.141892 (0.019090) with: {'optimizer': 'SGD'}
0.150636 (0.011994) with: {'optimizer': 'RMSprop'}
0.142687 (0.018213) with: {'optimizer': 'Adagrad'}
0.143482 (0.017365) with: {'optimizer': 'Adadelta'}
0.152623 (0.010826) with: {'optimizer': 'Adam'}
0.145469 (0.015401) with: {'optimizer': 'Adamax'}
0.155803 (0.011690) with: {'optimizer': 'Nadam'}


In [ ]:
#model, learn_rate, momentum, param_grid = gs3Spec(input_dim=X.shape[1], batch_size=bestBatchSize, epochs=bestEpochs)

In [ ]:
#grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
#grid_result = grid.fit(np.array(X), np.array(Y))

In [ ]:
##we get no useful info from gs3 except what would be best to use for sgd
#bestSGDLearn = grid_result.best_params_.get('learn_rate')
#bestSGDMomentum = grid_result.best_params_.get('momentum')
#printResult(grid_result)

In [83]:
###GS4
model, init_mode, param_grid = gs4Spec(input_dim=X.shape[1], batch_size=bestBatchSize, epochs=bestEpochs, optimizer=bestOptimizer)

In [84]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(np.array(X), np.array(Y))

In [85]:
#from gs4, we get the best init mode.
bestInitMode = grid_result.best_params_.get('init_mode')
printResult(grid_result)

Best: 0.158188 using {'init_mode': 'glorot_normal'}
0.155008 (0.010263) with: {'init_mode': 'uniform'}
0.154213 (0.011580) with: {'init_mode': 'lecun_uniform'}
0.154213 (0.009789) with: {'init_mode': 'normal'}
0.141892 (0.019090) with: {'init_mode': 'zero'}
0.158188 (0.010760) with: {'init_mode': 'glorot_normal'}
0.156598 (0.011726) with: {'init_mode': 'glorot_uniform'}
0.153816 (0.010707) with: {'init_mode': 'he_normal'}
0.158188 (0.012052) with: {'init_mode': 'he_uniform'}


In [ ]:
###GS5
model, activation, param_grid = gs5Spec(input_dim=X.shape[1], batch_size=bestBatchSize, epochs=bestEpochs, optimizer=bestOptimizer, init_mode=bestInitMode)

In [ ]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(np.array(X), np.array(Y))

In [ ]:
bestActivation = grid_result.best_params_.get('activation')
printResult(grid_result)

In [ ]:
###GS7
model, neurons, param_grid = gs7Spec(input_dim=X.shape[1], batch_size=bestBatchSize, epochs=bestEpochs, optimizer=bestOptimizer, init_mode=bestInitMode, dropout_rate = 0, weight_constraint=0)

In [ ]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(np.array(X), np.array(Y))

In [ ]:
bestNeurons = grid_result.best_params_.get('neurons')
printResult(grid_result)

In [ ]:
####Weather Dataset.
X, Y = weatherSetup()

In [ ]:
model, batch_size, epochs, param_grid = gs1Spec(input_dim=X.shape[1])

In [ ]:
#####Processing Search 1 - for Batch Size and Epochs.
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(np.array(X), np.array(Y))

In [ ]:
bestBatchSize = grid_result.best_params_.get('batch_size')
bestEpochs = grid_result.best_params_.get('epochs')
printResult(grid_result)

In [ ]:
#####Processing Search 2 - for best Optimizer.
model, optimizer, param_grid = gs2Spec(input_dim=X.shape[1], batch_size=bestBatchSize, epochs=bestEpochs)

In [ ]:
#####Processing Search 1 - for Batch Size and Epochs.
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(np.array(X), np.array(Y))

In [ ]:
#From GS2, we get the best optimizer.  Save this for future reference.  Pray it is not SGD.
bestOptimizer = grid_result.best_params_.get('optimizer')
printResult(grid_result)

In [ ]:
#model, learn_rate, momentum, param_grid = gs3Spec(input_dim=X.shape[1], batch_size=bestBatchSize, epochs=bestEpochs)

In [ ]:
#grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
#grid_result = grid.fit(np.array(X), np.array(Y))

In [ ]:
##we get no useful info from gs3 except what would be best to use for sgd
#bestSGDLearn = grid_result.best_params_.get('learn_rate')
#bestSGDMomentum = grid_result.best_params_.get('momentum')
#printResult(grid_result)

In [ ]:
###GS4
model, init_mode, param_grid = gs4Spec(input_dim=X.shape[1], batch_size=bestBatchSize, epochs=bestEpochs, optimizer=bestOptimizer)

In [ ]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(np.array(X), np.array(Y))

In [ ]:
#from gs4, we get the best init mode.
bestInitMode = grid_result.best_params_.get('init_mode')
printResult(grid_result)

In [ ]:
###GS5
model, activation, param_grid = gs5Spec(input_dim=X.shape[1], batch_size=bestBatchSize, epochs=bestEpochs, optimizer=bestOptimizer, init_mode=bestInitMode)

In [ ]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(np.array(X), np.array(Y))

In [ ]:
bestActivation = grid_result.best_params_.get('activation')
printResult(grid_result)

In [ ]:
###GS7
model, neurons, param_grid = gs7Spec(input_dim=X.shape[1], batch_size=bestBatchSize, epochs=bestEpochs, optimizer=bestOptimizer, init_mode=bestInitMode, dropout_rate = 0, weight_constraint=0)

In [ ]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(np.array(X), np.array(Y))

In [ ]:
bestNeurons = grid_result.best_params_.get('neurons')
printResult(grid_result)